In [1]:
#in powershell
%pip install spotipy duckdb python-dotenv pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## create cred.py  in the root directory 
add your own client id and secrets using the values you have optained from spotify.


### cred.py
```python
# get values from 
# https://developer.spotify.com/dashboard/
spotify_client_id = "your id"
spotify_client_secret = "your secret"
```


In [2]:
# import the credentials from cred.py
from cred import spotify_client_id, spotify_client_secret

In [3]:
import os
from dotenv import load_dotenv

import duckdb
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyOAuth

# custom functions
import functions as fn


In [4]:
# Load .env
load_dotenv()

CLIENT_ID = spotify_client_id
CLIENT_SECRET = spotify_client_secret
REDIRECT_URI = "http://127.0.0.1:8000/callback"
SCOPE = (
    "user-library-read playlist-read-private playlist-modify-private playlist-modify-public"
)

assert CLIENT_ID and CLIENT_SECRET and REDIRECT_URI, "Missing Spotify env vars"

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        redirect_uri=REDIRECT_URI,
        scope=SCOPE,
        open_browser=True,  # will open auth page in browser
        cache_path=".cache-spotifydb"  # token cache
    )
)

current_user = sp.current_user()
current_user["display_name"], current_user["id"]


('1257524228', '1257524228')

In [5]:
# In-memory DB (great for playing around)
# con = duckdb.connect(database=':memory:')

# If you want persistent on-disk:
con = duckdb.connect(database='spotify.duckdb')


In [24]:
df_tables = con.execute(f"SHOW TABLES").df()
display(df_tables)


,name
0,my_liked_songs


In [25]:

try:
    my_liked_songs = fn.load_my_saved_tracks(sp)
    display(f"My Liked Songs: {len(my_liked_songs)}")
    display(my_liked_songs.head())

    fn.df_to_duckdb(con, my_liked_songs, "my_liked_songs")
except Exception as e:
    print("Error loading My Liked Songs:", e)
    my_liked_songs = fn.duckdb_to_df(con, "my_liked_songs")


'My Liked Songs: 1734'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-12-02T21:16:26Z,3WwoUlqRzqd3fAnMH8CXWJ,Joe's Rogaine Experience,Joe's Rogaine Experience,5vFGbEPRn8lsNLqpkvvuPP,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,264192,False,13,False,spotify:track:3WwoUlqRzqd3fAnMH8CXWJ
1,2025-11-29T00:42:11Z,7mykoq6R3BArsSpNDjFQTm,I Really Want to Stay at Your House,"Cyberpunk 2077: Radio, Vol. 2 (Original Soundt...",1VGVJdmvOSRK2w9RKXk18A,"Rosa Walton, Hallie Coggins",1X0HaTcdkHW7LviblBiEeq,246652,False,78,False,spotify:track:7mykoq6R3BArsSpNDjFQTm
2,2025-11-22T17:14:10Z,3jaZi4JMoh1o5znSKNqIgI,Again,Again,1WxeeVTaJxXhORenFx0Xbk,"Shadow Realm, Meg Wills",2qybVy8UqYXDx6x6BZedPE,249390,False,25,False,spotify:track:3jaZi4JMoh1o5znSKNqIgI
3,2025-11-20T00:04:10Z,3ZrZYpxHQiBgDpfxHkTnLC,Elon's Musk,Elon's Musk,3BZNhRBQknND6A7jjfE8lT,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,220536,False,42,False,spotify:track:3ZrZYpxHQiBgDpfxHkTnLC
4,2025-10-12T21:43:37Z,1gR5TDDEvDkxTs1UGjR910,what i got - lofi,study and chill with sublime,696pAMUhtOEyenQukQKRxI,"lonelyboy, Sublime",30pF2pv50VXX6TgiMmKKjc,133333,False,21,False,spotify:track:1gR5TDDEvDkxTs1UGjR910


In [20]:
new_liked_songs = duckdb.query(
"""
select * from my_liked_songs 
order by saved_at 
desc limit 100
"""
).to_df()

display(f"new_liked_songs: {len(new_liked_songs)}")
display(new_liked_songs.head())

# let's save this as a Playlist on Spotify
playlist_id = fn.create_playlist(
    sp,
    name="**New Liked Songs",
    description="My 100 most recently liked songs, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, new_liked_songs["uri"].tolist())


'new_liked_songs: 100'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-12-02T21:16:26Z,3WwoUlqRzqd3fAnMH8CXWJ,Joe's Rogaine Experience,Joe's Rogaine Experience,5vFGbEPRn8lsNLqpkvvuPP,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,264192,False,13,False,spotify:track:3WwoUlqRzqd3fAnMH8CXWJ
1,2025-11-29T00:42:11Z,7mykoq6R3BArsSpNDjFQTm,I Really Want to Stay at Your House,"Cyberpunk 2077: Radio, Vol. 2 (Original Soundt...",1VGVJdmvOSRK2w9RKXk18A,"Rosa Walton, Hallie Coggins",1X0HaTcdkHW7LviblBiEeq,246652,False,78,False,spotify:track:7mykoq6R3BArsSpNDjFQTm
2,2025-11-22T17:14:10Z,3jaZi4JMoh1o5znSKNqIgI,Again,Again,1WxeeVTaJxXhORenFx0Xbk,"Shadow Realm, Meg Wills",2qybVy8UqYXDx6x6BZedPE,249390,False,25,False,spotify:track:3jaZi4JMoh1o5znSKNqIgI
3,2025-11-20T00:04:10Z,3ZrZYpxHQiBgDpfxHkTnLC,Elon's Musk,Elon's Musk,3BZNhRBQknND6A7jjfE8lT,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,220536,False,42,False,spotify:track:3ZrZYpxHQiBgDpfxHkTnLC
4,2025-10-12T21:43:37Z,1gR5TDDEvDkxTs1UGjR910,what i got - lofi,study and chill with sublime,696pAMUhtOEyenQukQKRxI,"lonelyboy, Sublime",30pF2pv50VXX6TgiMmKKjc,133333,False,21,False,spotify:track:1gR5TDDEvDkxTs1UGjR910


In [23]:
cream_of_crop = duckdb.query(
"""
select * from my_liked_songs 
order by popularity 
desc limit 100
"""
).to_df()

display(f"cream_of_crop: {len(cream_of_crop)}")
display(cream_of_crop.head())

# let's save this as a Playlist on Spotify
playlist_id = fn.create_playlist(
    sp,
    name="**Cream of Crop",
    description="My 100 most popular liked songs, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, cream_of_crop["uri"].tolist())

'cream_of_crop: 100'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-09-03T18:14:36Z,1CPZ5BxNNd0n0nF4Orb9JS,Golden,KPop Demon Hunters (Soundtrack from the Netfli...,14JkAa6IiFaOh5s0nMyMU9,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo...",2yNNYQBChuox9A5Ka93BIn,194607,False,99,False,spotify:track:1CPZ5BxNNd0n0nF4Orb9JS
1,2019-01-12T18:41:51Z,7BKLCZ1jbUBVqRi2FVlTVw,Closer,Closer,0rSLgV8p5FzfnqlEk4GzxE,"The Chainsmokers, Halsey",69GGBxA162lTqCwzJG5jLp,244960,False,87,False,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw
2,2019-09-21T18:43:08Z,6ECp64rv50XVz93WvxXMGF,This Love,Songs About Jane: 10th Anniversary Edition,5zClcGCSWj926AMjvBNSLc,Maroon 5,04gDigrS5kc9YWfZHwBETP,206200,False,87,False,spotify:track:6ECp64rv50XVz93WvxXMGF
3,2015-07-20T21:23:34Z,57bgtoPSgt236HzfBOd8kj,Thunderstruck,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,AC/DC,711MCceyCBcFnzjGY4Q7Un,292333,False,87,False,spotify:track:57bgtoPSgt236HzfBOd8kj
4,2016-09-30T23:35:05Z,3ZOEytgrvLwQaqXreDs2Jx,Can't Stop,By the Way (Deluxe Edition),6deiaArbeoqp1xPEGdEKp1,Red Hot Chili Peppers,0L8ExT028jH3ddEcZwqJJ5,269000,False,87,False,spotify:track:3ZOEytgrvLwQaqXreDs2Jx
